This notebook will help you to learn how YugabyteDB(YCQL) can connect from Azure Databricks. 
In this demo, following scenarios are covered

1. Load CSV from Databricks Filestore into YugabyteDB 
2. Import Data into YugabyteDB
   2a. Import Parquet file into YugabyteDB
   2b. Import Avro file into YugabyteDB
3. Export YugabyteDB Table into Storage Folder
   3a.  Export YugabyteDB table into Parquet file
   3b.  Export YugabyteDB table into Avro file
4. Import Delta Table into YugabyteDB table  
5. Export YugabyteDB table into Delta Lake Table
6. Data Visualization and sample queries

In [0]:
%python
#1. Load Salesrecords CSV directly into YugabyteDB (YCQL) Table
df1 = spark.read.format("csv").option("header", "true").option("inferschema", "true").load("dbfs:/FileStore/tables/1000_Sales_Records_1.csv")

import os
import pyspark
#import pyspark_cassandra
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import col
spark = (SparkSession.builder
    .appName('simple_etl')
    .config("spark.cassandra.connection.host", "10.14.16.7")
    .config('spark.cassandra.connection.port', '9042')
    .config("spark.cassandra.auth.username", "cassandra")
    .config("spark.cassandra.auth.password", "cassandra")
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
    .config("spark.cassandra.connection.localDC", "eastus")
    .getOrCreate())
df1.createOrReplaceTempView("salesorder")
pqrdf = spark.sql("SELECT orderid, region,country,itemtype,saleschannel,orderpriority,orderdate,shipdate,unitssold, unitprice, unitcost, totalrevenue, totalcost , totalprofit  FROM salesorder ")
pqrdf.write.format("org.apache.spark.sql.cassandra").mode("overwrite").option("confirm.truncate","true").options(table="salesprofit_test", keyspace ="demo").save()
#df1.write.format("org.apache.spark.sql.cassandra").mode('append').options(table="salesprofit_test1", keyspace="demo").save()

In [0]:
%python
#2a. Read parquet file and load into YugabyteDB
sourcedf = sqlContext.read.parquet("/tmp/salesprofit")
sourcedf.createOrReplaceTempView("salesorder")
df = spark.sql("SELECT orderid, region,country,itemtype,saleschannel,orderpriority,orderdate,shipdate,unitssold, unitprice, unitcost, totalrevenue, totalcost , totalprofit  FROM salesorder ")
df.write.format("org.apache.spark.sql.cassandra").mode("overwrite").option("confirm.truncate","true").options(table="salesprofit_test", keyspace ="demo").save()

In [0]:
%python
#2b. Read avro file and load into YugabyteDB
datafavro = spark.read.format("avro").load("/tmp/salesprofit_avro_export")
datafavro.createOrReplaceTempView("salesorder")
df = spark.sql("SELECT orderid, region,country,itemtype,saleschannel,orderpriority,orderdate,shipdate,unitssold, unitprice, unitcost, totalrevenue, totalcost , totalprofit  FROM salesorder ")
df.write.format("org.apache.spark.sql.cassandra").mode("overwrite").option("confirm.truncate","true").options(table="salesprofit_avro_test", keyspace ="demo").save()

In [0]:
%python
#3a. Export YugabyteDB table into parquet file format.
pd = spark.read.format("org.apache.spark.sql.cassandra").options(table="salesprofit_test", keyspace="demo").load()
pd.write.mode("overwrite").parquet("/tmp/export/salesprofit_test")

In [0]:
%python
#3a. Export YugabyteDB table into avro file format
pd = spark.read.format("org.apache.spark.sql.cassandra").options(table="salesprofit_avro_test", keyspace="demo").load()
pd.write.format('avro').mode('overwrite').save("/tmp/export/avro/salesprofit_test_avro")

In [0]:
%python
#4 Import data from Delta table to YugabyteDB (YCQL)
dataf = spark.read.format("delta").load("dbfs:/user/hive/warehouse/sample_table4")
dataf.write.format("org.apache.spark.sql.cassandra").mode("overwrite").option("confirm.truncate","true").options(table="employee", keyspace ="demo").save()


In [0]:
%python
#5 Read a table from YugabyteDB (YCQL) using JBDC and load into Delta table
pd = spark.read.format("org.apache.spark.sql.cassandra").options(table="salesprofit_test", keyspace="demo").load()
pd.write.format ( "delta" ).mode("overwrite").saveAsTable ( "sales_details" )

In [0]:
%python
#6. Data Visualization using YugabyteDB
df = spark.read.format("org.apache.spark.sql.cassandra").options(table="salesprofit_test", keyspace="demo").load()
df.show()
df.createOrReplaceTempView ( "profitanalysis" )

+---------+-------------+---------------+----------+-------------+--------------------+------------+----------+----------+-----------+------------+--------+---------+---------+
 orderid| country| itemtype| orderdate|orderpriority| region|saleschannel| shipdate| totalcost|totalprofit|totalrevenue|unitcost|unitprice|unitssold|
+---------+-------------+---------------+----------+-------------+--------------------+------------+----------+----------+-----------+------------+--------+---------+---------+
402646195| Vanuatu|Office Supplies| 3/1/2016| L|Australia and Oce...| Online| 3/28/2016| 426267.52| 102515| 528782.52| 524.96| 651.21| 812|
612943828| Pakistan| Meat| 2/17/2013| C|Middle East and N...| Online| 3/1/2013| 931418.26| 146088.8| 1077507.06| 364.69| 421.89| 2554|
298856723| Niger| Baby Food| 3/8/2017| H| Sub-Saharan Africa| Online| 4/3/2017| 1147824| 690192| 1838016| 159.42| 255.28| 7200|
409873998| Turkey| Cosmetics| 5/15/2010| C|Middle East and N...| Offline| 6/3/2010|2548771.07| 1682887.73| 4231658.8| 263.33| 437.2| 9679|
791975486| Serbia| Beverages| 2/27/2016| H| Europe| Offline| 3/20/2016| 63611.79| 31335.66| 94947.45| 31.79| 47.45| 2001|
419124829| Italy| Beverages| 8/30/2013| H| Europe| Offline| 9/19/2013| 229078.74| 112845.96| 341924.7| 31.79| 47.45| 7206|
936042296| Singapore| Beverages| 2/12/2010| L| Asia| Offline| 3/17/2010| 145312.09| 71581.86| 216893.95| 31.79| 47.45| 4571|
890695369| Cuba| Beverages| 1/9/2011| C|Central America a...| Offline| 2/23/2011| 171920.32| 84689.28| 256609.6| 31.79| 47.45| 5408|
964214932| Libya| Cereal| 3/27/2014| M|Middle East and N...| Offline| 3/31/2014| 173322.8| 131113.2| 304436| 117.11| 205.7| 1480|
683458888| Armenia| Cereal|11/24/2014| H| Europe| Online|12/28/2014|1115824.08| 844085.52| 1959909.6| 117.11| 205.7| 9528|
290455615| Israel| Personal Care| 5/13/2012| L|Middle East and N...| Online| 5/28/2012| 63810.42| 28217.56| 92027.98| 56.67| 81.73| 1126|
384013640| Malawi| Vegetables| 6/22/2012| L| Sub-Saharan Africa| Online| 7/19/2012| 638783.25| 443488.25| 1082271.5| 90.93| 154.06| 7025|
105390059|Cote d'Ivoire| Cereal| 7/30/2016| L| Sub-Saharan Africa| Online| 8/25/2016| 716127.65| 541727.85| 1257855.5| 117.11| 205.7| 6115|
722088277| Russia| Fruits| 7/3/2016| C| Europe| Online| 7/3/2016| 5688.24| 1981.02| 7669.26| 6.92| 9.33| 822|
334612929| Indonesia| Personal Care| 8/17/2016| H| Asia| Offline| 10/3/2016| 467867.52| 206895.36| 674762.88| 56.67| 81.73| 8256|
235702931| Mali| Cereal| 3/17/2012| L| Sub-Saharan Africa| Online| 4/3/2012| 1005974.9| 760988.1| 1766963| 117.11| 205.7| 8590|
645597255| Spain| Vegetables| 10/2/2012| C| Europe| Online|10/25/2012| 493658.97| 342732.77| 836391.74| 90.93| 154.06| 5429|
368547379| Tajikistan| Meat| 7/7/2012| M| Asia| Online| 7/12/2012| 78043.66| 12240.8| 90284.46| 364.69| 421.89| 214|
703815782| Switzerland| Meat|12/22/2012| C| Europe| Online| 1/7/2013|1307413.65| 205062| 1512475.65| 364.69| 421.89| 3585|
167787253| Palau| Vegetables| 6/29/2010| C|Australia and Oce...| Offline| 7/16/2010| 75653.76| 52524.16| 128177.92| 90.93| 154.06| 832|
+---------+-------------+---------------+----------+-------------+--------------------+------------+----------+----------+-----------+------------+--------+---------+---------+
only showing top 20 rows

In [0]:
%sql
SELECT * FROM profitanalysis 

orderid,country,itemtype,orderdate,orderpriority,region,saleschannel,shipdate,totalcost,totalprofit,totalrevenue,unitcost,unitprice,unitssold
905392587,Kiribati,Cereal,7/24/2012,L,Australia and Oceania,Offline,8/16/2012,543507.51,411146.19,954653.7,117.11,205.7,4641
613830459,Benin,Cereal,12/12/2016,C,Sub-Saharan Africa,Offline,1/16/2017,577118.08,436571.52,1013689.6,117.11,205.7,4928
927666509,Bahrain,Household,6/27/2012,H,Middle East and North Africa,Online,7/17/2012,3010214.6,992722.7,4002937.3,502.54,668.27,5990
736967885,Portugal,Cosmetics,2/27/2011,C,Europe,Offline,3/12/2011,1060956.57,700522.23,1761478.8,263.33,437.2,4029
432995069,Uzbekistan,Clothes,11/11/2010,C,Asia,Online,12/13/2010,61573.12,126169.92,187743.04,35.84,109.28,1718
247802054,Senegal,Household,8/27/2012,L,Sub-Saharan Africa,Offline,9/8/2012,4517332.06,1489746.97,6007079.03,502.54,668.27,8989
444604098,Belarus,Baby Food,10/26/2010,H,Europe,Offline,10/31/2010,1129968.96,679455.68,1809424.64,159.42,255.28,7088
221530139,Russia,Fruits,1/8/2011,L,Europe,Offline,1/26/2011,31458.32,10955.86,42414.18,6.92,9.33,4546
927232635,Bangladesh,Personal Care,10/15/2016,L,Asia,Online,11/24/2016,430521.99,190380.82,620902.81,56.67,81.73,7597
860886800,Sri Lanka,Snacks,11/13/2013,L,Asia,Offline,11/23/2013,427761.6,242064.6,669826.2,97.44,152.58,4390


In [0]:
%sql
SELECT region, sum(TotalRevenue) FROM profitanalysis group by region

region,sum(CAST(TotalRevenue AS DOUBLE))
Middle East and North Africa,1.7510653574E8
Australia and Oceania,1.056895726E8
Europe,3.5316746293E8
Sub-Saharan Africa,3.5672425012E8
Central America and the Caribbean,1.4399761051E8
Asia,1.6767480949000004E8
North America,2.496159894E7
